In [190]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.metrics  import silhouette_score
from scipy.spatial.distance import cdist
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [236]:
movie_for_ml = pd.read_csv('data/movies_for_ml.csv.zip')

In [267]:
cols = movie_for_ml.columns[2:]
X = movie_for_ml[cols]

scaler = StandardScaler().fit(X)
X_scaled = scaler.transform(X)

In [268]:
movie_scaled = pd.DataFrame()


In [269]:
movie_scaled['tconst']=movie_for_ml['tconst']
movie_scaled['originalTitle']=movie_for_ml['originalTitle']
movie_scaled

,tconst,originalTitle
0,tt0008879,Berg-Ejvind och hans hustru
1,tt0009893,Die Austernprinzessin
2,tt0009937,Blind Husbands
3,tt0009968,Broken Blossoms or The Yellow Man and the Girl
4,tt0010247,Herr Arnes pengar
...,...,...
11636,tt9882084,Chasing Happiness
11637,tt9886872,Munthiri Monchan
11638,tt9900782,Kaithi
11639,tt9902160,Herself


In [270]:
scaled_df = pd.DataFrame(X_scaled,columns=cols)
scaled_df

,startYear,nm0000005,nm0000008,nm0000019,nm0000033,nm0000036,nm0000037,nm0000040,nm0000041,nm0000045,...,Music,Musical,Mystery,News,Romance,Sci-Fi,Sport,Thriller,War,Western
0,-2.973333,-0.047313,-0.009269,-0.039353,-0.049974,-0.020729,-0.020729,-0.030754,-0.047313,-0.009269,...,-0.219266,-0.149352,-0.244175,-0.044494,-0.463280,-0.149952,-0.160966,-0.318642,-0.214928,-0.116546
1,-2.933287,-0.047313,-0.009269,-0.039353,-0.049974,-0.020729,-0.020729,-0.030754,-0.047313,-0.009269,...,-0.219266,-0.149352,-0.244175,-0.044494,-0.463280,-0.149952,-0.160966,-0.318642,-0.214928,-0.116546
2,-2.933287,-0.047313,-0.009269,-0.039353,-0.049974,-0.020729,-0.020729,-0.030754,-0.047313,-0.009269,...,-0.219266,-0.149352,-0.244175,-0.044494,2.158521,-0.149952,-0.160966,-0.318642,-0.214928,-0.116546
3,-2.933287,-0.047313,-0.009269,-0.039353,-0.049974,-0.020729,-0.020729,-0.030754,-0.047313,-0.009269,...,-0.219266,-0.149352,-0.244175,-0.044494,2.158521,-0.149952,-0.160966,-0.318642,-0.214928,-0.116546
4,-2.933287,-0.047313,-0.009269,-0.039353,-0.049974,-0.020729,-0.020729,-0.030754,-0.047313,-0.009269,...,-0.219266,-0.149352,-0.244175,-0.044494,-0.463280,-0.149952,-0.160966,-0.318642,-0.214928,-0.116546
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11636,1.071389,-0.047313,-0.009269,-0.039353,-0.049974,-0.020729,-0.020729,-0.030754,-0.047313,-0.009269,...,4.560661,-0.149352,-0.244175,-0.044494,-0.463280,-0.149952,-0.160966,-0.318642,-0.214928,-0.116546
11637,1.071389,-0.047313,-0.009269,-0.039353,-0.049974,-0.020729,-0.020729,-0.030754,-0.047313,-0.009269,...,-0.219266,-0.149352,-0.244175,-0.044494,2.158521,-0.149952,-0.160966,-0.318642,-0.214928,-0.116546
11638,1.071389,-0.047313,-0.009269,-0.039353,-0.049974,-0.020729,-0.020729,-0.030754,-0.047313,-0.009269,...,-0.219266,-0.149352,-0.244175,-0.044494,-0.463280,-0.149952,-0.160966,3.138315,-0.214928,-0.116546
11639,1.111436,-0.047313,-0.009269,-0.039353,-0.049974,-0.020729,-0.020729,-0.030754,-0.047313,-0.009269,...,-0.219266,-0.149352,-0.244175,-0.044494,-0.463280,-0.149952,-0.160966,-0.318642,-0.214928,-0.116546


In [ ]:
movie_scaled_concat = pd.concat([movie_scaled,scaled_df],axis=1)

In [262]:
cols = movie_scaled_concat.columns[2:]
X = movie_scaled_concat[cols]

modelKM = KMeans(n_clusters=10).fit(X)

movie_scaled_concat['cluster']=modelKM.labels_

In [263]:
movie_scaled_concat[movie_scaled_concat['originalTitle'].str.contains('Batman')]

,tconst,originalTitle,startYear,nm0000005,nm0000008,nm0000019,nm0000033,nm0000036,nm0000037,nm0000040,...,Musical,Mystery,News,Romance,Sci-Fi,Sport,Thriller,War,Western,cluster
3860,tt0096895,Batman,-0.130014,-0.047313,-0.009269,-0.039353,-0.049974,-0.020729,-0.020729,-0.030754,...,-0.149352,-0.244175,-0.044494,-0.46328,-0.149952,-0.160966,-0.318642,-0.214928,-0.116546,2
4110,tt0103776,Batman Returns,-0.009873,-0.047313,-0.009269,-0.039353,-0.049974,-0.020729,-0.020729,-0.030754,...,-0.149352,-0.244175,-0.044494,-0.46328,-0.149952,-0.160966,-0.318642,-0.214928,-0.116546,2
4219,tt0106364,Batman: Mask of the Phantasm,0.030174,-0.047313,-0.009269,-0.039353,-0.049974,-0.020729,-0.020729,-0.030754,...,-0.149352,-0.244175,-0.044494,-0.46328,-0.149952,-0.160966,-0.318642,-0.214928,-0.116546,2
6241,tt0372784,Batman Begins,0.510735,-0.047313,-0.009269,-0.039353,-0.049974,-0.020729,-0.020729,-0.030754,...,-0.149352,-0.244175,-0.044494,-0.46328,-0.149952,-0.160966,-0.318642,-0.214928,-0.116546,2
10166,tt4116284,The Lego Batman Movie,0.991296,-0.047313,-0.009269,-0.039353,-0.049974,-0.020729,-0.020729,-0.030754,...,-0.149352,-0.244175,-0.044494,-0.46328,-0.149952,-0.160966,-0.318642,-0.214928,-0.116546,2
10811,tt6067832,Batman & Bill,0.991296,-0.047313,-0.009269,-0.039353,-0.049974,-0.020729,-0.020729,-0.030754,...,-0.149352,4.095427,-0.044494,-0.46328,-0.149952,-0.160966,-0.318642,-0.214928,-0.116546,2


In [264]:
movie_scaled_concat['cluster'].value_counts()

2    11586
0       20
3       16
9        8
6        5
5        2
8        1
7        1
1        1
4        1
Name: cluster, dtype: int64

In [260]:
film = movie_scaled_concat[movie_scaled_concat['originalTitle']=='Batman Begins']
n_cluster = movie_scaled_concat['cluster'][movie_scaled_concat['originalTitle']=='Batman Begins'].values
df_for_KNN=movie_scaled_concat[movie_scaled_concat['cluster']==n_cluster[0]]

In [255]:
cols = movie_scaled_concat.columns[2:-1]
X = df_for_KNN[cols]

distanceKNN = NearestNeighbors(n_neighbors=10).fit(X)

In [256]:
list_film = distanceKNN.kneighbors(film[cols].values)
items = list_film[1].reshape(1,10)[0]

In [258]:
suggestion = movie_for_ml.iloc[items]
suggestion

,tconst,originalTitle,startYear,nm0000005,nm0000008,nm0000019,nm0000033,nm0000036,nm0000037,nm0000040,...,Music,Musical,Mystery,News,Romance,Sci-Fi,Sport,Thriller,War,Western
4433,tt0112445,Badkonake sefid,1995,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4715,tt0119346,I Married a Strange Person!,1997,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4714,tt0119282,Hercules,1997,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5736,tt0282778,Nadodikkattu,1987,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6784,tt0459505,Pitbull,2005,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7341,tt0978759,Frozen River,2008,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8783,tt1753549,Buck,2011,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4016,tt0101414,Beauty and the Beast,1991,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4211,tt0106305,La ardilla roja,1993,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3780,tt0094898,Coming to America,1988,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
